# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
file_path = "/Users/zay/Desktop/Boot Camp/Module 6/python-api-challenge/"

city_data_df = pd.read_csv(f"{file_path}/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,porto franco,-6.3383,-47.3992,24.94,90,100,0.68,BR,1731621160
1,1,port-aux-francais,-49.3500,70.2167,2.95,74,81,10.58,TF,1731621161
2,2,nanao,37.0500,136.9667,14.10,89,100,1.21,JP,1731621162
3,3,saldanha,-33.0117,17.9442,17.03,88,90,6.64,ZA,1731621163
4,4,onoda,34.0014,131.1836,18.01,88,100,3.09,JP,1731621164


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
humidity = city_data_df["Humidity"].astype(int)
map_plot_1 = city_data_df.hvplot.points("Lng",
                                        "Lat",
                                        geo=True,
                                        tiles="OSM",
                                        size='Humidity',
                                        frame_width=800,
                                        frame_height=600,
                                        title="City Humidity Map")

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) 
                                        & (city_data_df["Max Temp"] > 21)
                                        & (city_data_df["Wind Speed"] < 4.5)
                                        & (city_data_df["Cloudiness"] < 3)]

# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
140,140,warmbad,-28.4500,18.7333,26.52,12,0,2.85,NaN,1731621329
231,231,al ghayzah,16.2079,52.1760,24.72,62,1,0.85,YE,1731621437
324,324,sirte,31.2089,16.5887,21.26,72,0,4.27,LY,1731621551
350,350,chabahar,25.2919,60.6430,26.32,64,2,1.54,IR,1731621582
402,402,san patricio,28.0170,-97.5169,25.55,21,0,3.46,US,1731621647
412,412,timimoun,29.2639,0.2310,21.21,24,0,2.06,DZ,1731621659
429,429,dwarka,22.2394,68.9678,26.55,58,2,4.17,IN,1731621682
466,466,jaisalmer,26.9147,70.9181,24.31,32,0,4.34,IN,1731621730
483,483,bay minette,30.8830,-87.7730,23.14,46,0,2.83,US,1731621688
548,548,dapchi,12.4904,11.4944,24.30,14,0,3.96,NG,1731621829


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
140,warmbad,NaN,-28.4500,18.7333,12,
231,al ghayzah,YE,16.2079,52.1760,62,
324,sirte,LY,31.2089,16.5887,72,
350,chabahar,IR,25.2919,60.6430,64,
402,san patricio,US,28.0170,-97.5169,21,
412,timimoun,DZ,29.2639,0.2310,24,
429,dwarka,IN,22.2394,68.9678,58,
466,jaisalmer,IN,26.9147,70.9181,32,
483,bay minette,US,30.8830,-87.7730,46,
548,dapchi,NG,12.4904,11.4944,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000 #metres
params = {"apiKey" : geoapify_key,
          "categories" : "accommodation.hotel"}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
warmbad - nearest hotel: No hotel found
al ghayzah - nearest hotel: فندق تاج العرب
sirte - nearest hotel: فندق المدينة-سرت
chabahar - nearest hotel: هتل سپیده
san patricio - nearest hotel: No hotel found
timimoun - nearest hotel: أناف
dwarka - nearest hotel: The Dwarika Hotel
jaisalmer - nearest hotel: Jeet Mahal
bay minette - nearest hotel: No hotel found
dapchi - nearest hotel: No hotel found
houma - nearest hotel: Courtyard Houma
taoudenni - nearest hotel: No hotel found
west livingston - nearest hotel: Best Western Plus
al burayqah - nearest hotel: فندق ماس ليبيا


,City,Country,Lat,Lng,Humidity,Hotel Name
140,warmbad,NaN,-28.4500,18.7333,12,No hotel found
231,al ghayzah,YE,16.2079,52.1760,62,فندق تاج العرب
324,sirte,LY,31.2089,16.5887,72,فندق المدينة-سرت
350,chabahar,IR,25.2919,60.6430,64,هتل سپیده
402,san patricio,US,28.0170,-97.5169,21,No hotel found
412,timimoun,DZ,29.2639,0.2310,24,أناف
429,dwarka,IN,22.2394,68.9678,58,The Dwarika Hotel
466,jaisalmer,IN,26.9147,70.9181,32,Jeet Mahal
483,bay minette,US,30.8830,-87.7730,46,No hotel found
548,dapchi,NG,12.4904,11.4944,14,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)